In [1]:
!curl "https://storage.googleapis.com/kaggle-data-sets/10449/44567/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1589104146&Signature=nFzhdgwd2FjSSGLCV%2FpouCSmA%2FjNr9zst8%2FlvrFeYyNjs6Rs2HTeSiRJi8tQLWBjeqC3%2BZmJ9kbl7kez2rmGCj8F4TsQPyu9aCNg77TZFjh8YCrZQPsSvlfswTY2PnrJrH5JHIoQOSnISD8WaVGQWdyGbRArJrHBX%2B5cILPAhT6RD%2BXPOqWNJDzxjEsUzl2NZlov4Vg1rVEHsA0ZM7mU43q8uHxKCGo5t1T9PXywJ0nft6mpIDvCeH11RhJJu33QovEiHsBA%2BDa8Mj4XntHos21RKEgsLPikjNCXMMQ2JWvLpJrmC0rmw2xUJXhh1h%2BUvy%2BHPGm3ItZMgD%2BOK0%2B%2FYA%3D%3D&response-content-disposition=attachment%3B+filename%3D10-monkey-species.zip" -H "User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:75.0) Gecko/20100101 Firefox/75.0" -H "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8" -H "Accept-Language: en-US,en;q=0.5" --compressed -o monkey_breed.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  547M  100  547M    0     0  1521k      0  0:06:08  0:06:08 --:--:-- 1450k0  139k    0     0   124k      0  1:14:52  0:00:01  1:14:51  124k  0     0  1845k      0  0:05:03  0:00:04  0:04:59 1844k1M    0     0  1699k      0  0:05:29  0:03:38  0:01:51 1794k   71  389M    0     0  1703k      0  0:05:28  0:03:54  0:01:34 1739k  530M    0     0  1520k      0  0:06:08  0:05:57  0:00:11 1920k


In [2]:
!unzip monkey_breed.zip

Archive:  monkey_breed.zip
  inflating: monkey_labels.txt       
  inflating: training/training/n0/n0018.jpg  
  inflating: training/training/n0/n0019.jpg  
  inflating: training/training/n0/n0020.jpg  
  inflating: training/training/n0/n0021.jpg  
  inflating: training/training/n0/n0022.jpg  
  inflating: training/training/n0/n0023.jpg  
  inflating: training/training/n0/n0024.jpg  
  inflating: training/training/n0/n0025.jpg  
  inflating: training/training/n0/n0026.jpg  
  inflating: training/training/n0/n0027.jpg  
  inflating: training/training/n0/n0028.jpg  
  inflating: training/training/n0/n0029.jpg  
  inflating: training/training/n0/n0030.jpg  
  inflating: training/training/n0/n0031.jpg  
  inflating: training/training/n0/n0032.jpg  
  inflating: training/training/n0/n0033.jpg  
  inflating: training/training/n0/n0034.jpg  
  inflating: training/training/n0/n0035.jpg  
  inflating: training/training/n0/n0036.jpg  
  inflating: training/training/n0/n0037.jpg  
  inflating: tr

In [7]:
from keras.applications import MobileNet

img_rows, img_cols = 224, 224

MobileNet = MobileNet(weights="imagenet",
                     include_top=False,
                     input_shape=(img_rows, img_cols, 3))

for layer in MobileNet.layers:
    layer.trainable = False

for (i, layer) in enumerate(MobileNet.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [8]:
def addTopMobileNet(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation="relu")(top_model)
    top_model = Dense(1024, activation="relu")(top_model)
    top_model = Dense(512, activation="relu")(top_model)
    top_model = Dense(num_classes, activation="softmax")(top_model)
    return top_model

In [10]:
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten, ZeroPadding2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
import numpy as np

In [11]:
num_classes = 10 #there are 10 different monkey species
fc_head = addTopMobileNet(MobileNet, num_classes)

model = Model(inputs=MobileNet.inputs, outputs=fc_head)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [18]:
train_data_dir = "./training/training"
validation_data_dir = "./validation/validation"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest"
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = "categorical"
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = "categorical"
)

Found 1100 images belonging to 10 classes.
Found 273 images belonging to 10 classes.


In [13]:
nb_train_samples = 1100
nb_validation_samples = 273

In [19]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

opt = RMSprop(learning_rate=0.001)

checkpoint = ModelCheckpoint(
    "monkey_breed_classifier.h5",
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    verbose=1
)

earlystopping = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    restore_best_weights=True,
    verbose=1
)

callbacks = [earlystopping, checkpoint]

model.compile(loss="categorical_crossentropy",
             optimizer=RMSprop(lr=0.001),
             metrics=["accuracy"])

In [20]:
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
68/68 [==============================] - 186s 3s/step - loss: 1.8896 - accuracy: 0.4831 - val_loss: 1.1708 - val_accuracy: 0.6900

Epoch 00001: val_loss improved from inf to 1.17077, saving model to monkey_breed_classifier.h5
Epoch 2/5
68/68 [==============================] - 153s 2s/step - loss: 0.7021 - accuracy: 0.7982 - val_loss: 0.1265 - val_accuracy: 0.9300

Epoch 00002: val_loss improved from 1.17077 to 0.12647, saving model to monkey_breed_classifier.h5
Epoch 3/5
68/68 [==============================] - 152s 2s/step - loss: 0.5580 - accuracy: 0.8422 - val_loss: 0.0922 - val_accuracy: 0.9455

Epoch 00003: val_loss improved from 0.12647 to 0.09216, saving model to monkey_breed_classifier.h5
Epoch 4/5
68/68 [==============================] - 161s 2s/step - loss: 0.4703 - accuracy: 0.8627 - val_loss: 1.0425 - val_accuracy: 0.8424

Epoch 00004: val_loss did not improve from 0.09216
Epoch 5/5
68/68 [==============================] - 157s 2s/step - loss: 0.4390 - accuracy: 0

In [21]:
from keras.models import load_model
classifier = load_model('monkey_breed_classifier.h5')

In [22]:
import os
import cv2
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "mantled_howler ", 
                      "[1]": "patas_monkey",
                      "[2]": "bald_uakari",
                      "[3]": "japanese_macaque",
                      "[4]": "pygmy_marmoset ",
                      "[5]": "white_headed_capuchin",
                      "[6]": "silvery_marmoset",
                      "[7]": "common_squirrel_monkey",
                      "[8]": "black_headed_night_monkey",
                      "[9]": "nilgiri_langur"}

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}

In [35]:
def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,20):
    input_im = getRandomImage("./validation/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - bald_uakari
Class - bald_uakari
Class - nilgiri_langur
Class - silvery_marmoset
Class - silvery_marmoset
Class - black_headed_night_monkey
Class - common_squirrel_monkey
Class - black_headed_night_monkey
Class - common_squirrel_monkey
Class - white_headed_capuchin
Class - pygmy_marmoset 
Class - black_headed_night_monkey
Class - nilgiri_langur
Class - nilgiri_langur
Class - bald_uakari
Class - pygmy_marmoset 
Class - japanese_macaque
Class - pygmy_marmoset 
Class - common_squirrel_monkey
Class - white_headed_capuchin
